In this notebook, we're following the [Tensorflow for Poets](https://codelabs.developers.google.com/codelabs/tensorflow-for-poets/#0) tutorial, using Python to make an archaeological image classifier. This notebook + binder already has tensorflow installed. If you were working on your own machine, you'd install tensorflow with 

```pip install --upgrade "tensorflow==1.7.*"```



We need to get some training data, organized like so (imagining we were building a classifier that recognized Roman pottery types):
```
|
|-training-images
  |
  |-terrasig
  |-african_red_slip
  |-veranice_nera
```

We'll write a quick scraper that will retrieve images of Roman pottey from the [Atlas of Roman Pottery](http://potsherd.net/atlas/potsherd), using a couple of Python modules built for the task:

In [6]:
from bs4 import BeautifulSoup
import csv
import requests

r = requests.get('http://potsherd.net/atlas/Ware/')
soup = BeautifulSoup(r.content, "html.parser")

ModuleNotFoundError: No module named 'bs4'

In [5]:
!pip install bs4

  Running setup.py bdist_wheel for bs4 ... done
  Stored in directory: /Users/shawngraham/Library/Caches/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
